<a href="https://colab.research.google.com/github/51Sirius/BonusTrackProject/blob/main/script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Скрипт для генерации запросов заполнения таблиц
---

#### Имопрт библиотек
random - стандартная библотека рандома

In [1]:
!pip install Faker
import random
import datetime
from faker import Faker
from google.colab import files
from faker.providers.address.ru_RU import Provider

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 5.0 MB/s eta 0:00:00


#### Код для генерации случайных значений по типу, переменной

In [2]:
fake = Faker('ru_RU')
fake.add_provider(Provider)

def get_random_value(value_type):
  match value_type:
    case "callname":
      return fake.text().split()[0]
    case "id_p":
      raise ValueError
    case "first-name":
      return fake.first_name()
    case "last-name":
      return fake.last_name()
    case "surname":
      return fake.first_name() + 'ич'
    case "address":
      return fake.address()
    case "id":
      return random.randint(1, 100)
    case "text":
      return fake.text()
    case "birthdate":
      start_date = datetime.date(1974, 1, 1)
      end_date = datetime.date(2023, 12, 31)
      random_date = start_date + datetime.timedelta(days=random.randint(0, (end_date - start_date).days))
      return f"'{random_date.strftime('%Y-%m-%d')}'"
    case "price":
      return round(fake.pyfloat(right_digits=2, min_value=0, max_value=1000), 2)
    case "float":
      return round(fake.pyfloat(right_digits=2, min_value=100, max_value=10000), 2)
    case "time":
      return f'{random.randint(12, 24)}'
    case "bool":
      return f'{random.randint(0, 1)}'

#### Класс таблицы с функцией генерации запросов для неё

In [3]:
class Table:
  def __init__(self, tablename: str, columns: dict, count: int):
    self.tablename = tablename
    self.columns = columns
    self.count = count
    self.column_names = ', '.join(f'"{column}"' for column in list(self.columns.keys()))

  def generate(self) -> list:
    result = []
    for i in range(self.count):
          insert_query = f'INSERT INTO "{self.tablename}" ({self.column_names}) VALUES ({self.__generate_values(i)});\n'
          result.append(insert_query)
    return result

  def __generate_values(self, index) -> str:
    result = []
    for value_type in self.columns.values():
      try:
        value = get_random_value(value_type)
      except ValueError:
        value = index
      result.append(value)
    return ', '.join(str(value) for value in result)


#### Класс генератор файла с запросами для множества таблиц

In [4]:
class FileGenerator:
  def __init__(self, filename: str, tables: list):
    self.filename=filename
    self.tables = tables

  def create_file(self, count):
    passfilename = f'{self.filename}.sql'
    with open(self.filename, 'w') as f:
      for query in self.__create_all:
          f.write(query)
      f.close()

  def __create_all(self) -> list:
    result = []
    for table in self.tables:
      result += table.generate()
    return result

#### Заполнение таблиц

Таблица с типами свойств:

In [5]:
column_properties_type = {
    "Name" : "callname",
    "PropertiesId": "id"
}

table_properties_type = Table("PropertiesType", column_properties_type, 10)

In [6]:
column_properties = {
    "Id" : "id_p",
    "Name" : "callname",
    "ProductionSubCategoryId": "id"
}

table_properties = Table("Properties", column_properties, 10)

In [7]:
column_product_category = {
    "Id" : "id_p",
    "Name" : "callname",
}

table_properties = Table("ProductCategory", column_product_category, 10)

In [8]:
column_product_sub_category = {
    "Id" : "id_p",
    "Name" : "callname",
    "ProductCategoryId" : "id"
}

table_product_sub_category = Table("ProductSubCategory", column_product_category, 10)